In [5]:
import torch
import os
from sentence_transformers import SentenceTransformer
import qdrant_client
from qdrant_client.http.models import PointStruct, VectorParams, Distance

# Langkah 1: Membaca semua file .txt dalam satu direktori
directory_path = '/home/a_daffa_hilmi/documents/Dockerize/OCR/result_ocr'
peraturan_text = ""

for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            peraturan_text += file.read() + '\n\n'  # Menambahkan pemisah antar file

# Langkah 2: Inisialisasi Qdrant dan model Sentence Transformers
client = qdrant_client.QdrantClient("http://10.12.9.105:6333")

# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device='cuda' if torch.cuda.is_available() else 'cpu')

# Fungsi untuk mendapatkan embedding dari teks
def get_embeddings(text):
    return model.encode(text)

# Membagi teks menjadi paragraf dan mendapatkan embeddingnya
paragraphs = peraturan_text.split('\n\n')
embeddings = [get_embeddings(p) for p in paragraphs]

# Langkah 3: Menyimpan paragraf dan embeddingnya ke Qdrant
client.recreate_collection(
    collection_name="peraturan_perusahaan",
    vectors_config=VectorParams(size=len(embeddings[0]), distance=Distance.COSINE)
)

points = [
    PointStruct(id=i, vector=embeddings[i], payload={"text": paragraphs[i]})
    for i in range(len(paragraphs))
]

client.upsert(
    collection_name="peraturan_perusahaan",
    wait=True,
    points=points
)

# Fungsi untuk mencari paragraf yang relevan berdasarkan pertanyaan
def search_peraturan(query):
    query_vector = get_embeddings(query)
    results = client.search(
        collection_name="peraturan_perusahaan",
        query_vector=query_vector,
        limit=3
    )
    # Mengurutkan hasil berdasarkan skor relevansi
    # sorted_results = sorted(results, key=lambda x: x.score, reverse=True)
    return [res.payload["text"] for res in results]

# Contoh penggunaan
query = "seperti apa sistem pembayaran publikasi online?"
results = search_peraturan(query)
for result in results:
    print(result)

/tmp/ipykernel_782558/1777791300.py:32: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


PENJELASAN PEDOMAN PELAKSANAAN KERJA (CHARTER) KOMITE NOMINASI DAN REMUNERASI PERUSAHAAN PERSEROAN (PERSERO) PT TELEKOMUNIKASIINDONESIA, Tbk
1. LATAR BELAKANG Perusahaan Perseroan (Persero) PT Telekomunikasi Indonesia; Tbk selanjutnya disebut Perseroan , merupakan Badan Usaha Milik Negara yang bergerak dalam bidang telekomunikasi dengan produk digital connectivity, digital platform dan digital services. Perseroan saat ini telah terdaftar di pasar modal baik di dalam negeri maupun di luar negeri (New York Stock Exchange): Dalam menjalankan fungsi dan tugasnya di bidang nominasi dan remunerasi sesuai dengan ketentuan peraturan perundang-undangan yang beriaku; Dewan Komisaris dibantu oleh Komite Nominasi dan Remunerasi yang selanjutnya disebut Komite. Agar pelaksanaan tugas Komite dapat berjalan secara lebih efektif dan dapat dipertanggung jawabkan secara profesional, maka dalam pelaksanaan tugasnya Komite berpedoman Pedoman Pelaksanaan Kerja (Charter) Komite. pada
Peraturan Otoritas Jasa

In [2]:
# Contoh penggunaan
query = "apa itu pakta integritas?"
results = search_peraturan(query)
for result in results:
    print(result)